In [5]:
import numpy as np
import pandas as pd

## Load in data

In [6]:
from sklearn.datasets import fetch_california_housing

In [7]:
data = fetch_california_housing()

In [8]:
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [9]:
targets = data.target #scaled from 100,000

In [10]:
df = pd.DataFrame(data=data.data, columns=data.feature_names)

In [11]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [12]:
df.to_csv("median_housing_cost_data.tsv", sep="\t")

In [8]:
features = df.to_numpy()

In [9]:
targets.shape

(20640,)

In [10]:
features.shape

(20640, 8)

In [11]:
# Different regression tasks and analyze the complexity and other required components

# Can compare: linear regression vs. bayesian learning (maximim a posteriori or mle)


In [12]:
def linear_regression(X, y):
    
    I = np.eye(len(X[0]))    
    
    # compute parameter coefficients w
    moore_pen = np.linalg.inv(X.T@X)@X.T
    w = moore_pen@y
    
    # model prediction
    pred = X@w
    
    # residual error
    err = y - pred
    
    
    return w, pred, err
    

In [13]:
def regression_test(X, w):
    
    # predict output using passed data and parameters
    y = X@w
    
    return y
    

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_X, test_X, train_y, test_y = train_test_split(features, targets)

In [16]:
w, pred, err = linear_regression(train_X, train_y)

In [17]:
predictions = regression_test(test_X, w)

In [18]:
from sklearn.metrics import r2_score

In [19]:
from scipy.stats import linregress
_, _, r, _, _ = linregress(np.sort(test_y), np.sort(predictions))
r**2

0.9009436263404009

In [20]:
# MLE for this data: distribution multivariate gaussian ???

In [21]:
X_train_mean = np.mean(train_X, axis=0)
X_train_cov = np.cov(train_X.T)

In [81]:
X_test_demeaned = test_X - X_train_mean

In [93]:
X_test_demeaned[0].reshape(-1,1).T @ X_train_cov @ X_test_demeaned[0].reshape(-1,1)

array([[3.90082534e+11]])

In [22]:
X_train_mean.shape #sanity check

(8,)

In [24]:
X_train_cov.shape #sanity check

(8, 8)

In [51]:
train_X.shape

(15480, 8)

In [52]:
X_train_mean.shape

(8,)

In [54]:
X_train_demeaned = train_X - X_train_mean
X_train_demeaned.shape

(15480, 8)

In [58]:
d = X_train_mean.shape[0]

X_train_demeaned = train_X[:] - X_train_mean
X_train_cov_inv = np.linalg.inv(X_train_cov)
MVG_const = (1 / np.sqrt(np.power(2*np.pi, d) * np.abs(X_train_cov)) )
probabilities = MVG_const * np.exp(X_train_demeaned[0].T @ X_train_cov_inv @ X_train_demeaned[0])

In [59]:
probabilities.shape

(8, 8)

In [77]:
(train_X - X_train_mean).shape

(15480, 8)

In [32]:
train_X.shape

(15480, 8)

In [95]:
import scipy.stats as stats
X_train_datalikelihoods = stats.multivariate_normal.pdf(train_X, mean=X_train_mean, cov=X_train_cov)
X_test_datalikelihoods = stats.multivariate_normal.pdf(test_X, mean=X_train_mean, cov=X_train_cov)

In [94]:
X_train_datalikelihoods

array([1.55074251e-09, 1.78810619e-09, 1.89724525e-09, ...,
       1.53004923e-09, 3.79446726e-10, 7.43740009e-11])

In [96]:
X_test_datalikelihoods

array([9.97848033e-10, 8.43832693e-10, 5.65017528e-10, ...,
       1.13868693e-09, 1.82486412e-10, 1.02450358e-09])

In [66]:
train_X.shape

(15480, 8)

In [69]:
params = X_train_datalikelihoods @ train_X
params

array([ 3.46308640e-05,  2.88527568e-04,  4.76740315e-05,  9.82273614e-06,
        1.24401105e-02,  2.84974027e-05,  3.28990605e-04, -1.11820522e-03])

In [68]:
(X_train_datalikelihoods @ train_X).shape

(8,)

In [74]:
predz = test_X @ params
predz

array([24.74442343,  9.29298809,  9.06654669, ..., 21.23586815,
       21.31166661, 13.45527254])

In [75]:
test_y

array([1.578, 3.171, 2.095, ..., 2.432, 2.361, 1.538])

In [72]:
train_y.shape

(15480,)

In [76]:
_, _, r, _, _ = linregress(np.sort(test_y), np.sort(predz))
r**2

0.7608753414661369